In [1]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob
from read_xml import*
import tensorflow as tf


C:\Users\Robotmedia9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
image_data=[]
image_data=read_image()
xml_data=[]
xml_data=read_xml()

In [3]:
def switch(x):
    return {
        'person': 0,
        'bird': 1,
        'cat': 2,
        'cow': 3,
        'dog': 4,
        'horse': 5,
        'sheep': 6,
        'aeroplane': 7,
        'bicycle': 8,
        'boat': 9,
        'bus': 10,
        'car': 11,
        'motorbike': 12,
        'train': 13,
        'bottle': 14,
        'chair': 15,
        'diningtable': 16,
        'pottedplant': 17,
        'sofa': 18,
        'tvmonitor': 19
    }.get(x, 9) #default

In [4]:
#generate y label
name=[]
y_data=[]
for i in range(len(xml_data)):
    tmp=xml_data[i].findtext("object/name")
    tmp=str(tmp)
    name.append(tmp)
    tt=switch(tmp)
    y_data.append(tt)
# # 바꾸기 전과 바꾼후 비교    
# print(len(y_data))
# for i in range(100):
#     print(name[i] , y_data[i])
    

In [5]:
size=224
X = tf.placeholder(tf.float32, [None, size,size,3]) # image data 
Y = tf.placeholder(tf.float32, [None,20]) # image data 
W1 = tf.Variable(tf.random_normal([7, 7, 3, 96], stddev=0.01),name='W1')
W2 = tf.Variable(tf.random_normal([5, 5, 96, 256], stddev=0.01),name='W2')
W3 = tf.Variable(tf.random_normal([3, 3, 256, 384], stddev=0.01),name='W3')
W4 = tf.Variable(tf.random_normal([3, 3, 384, 384], stddev=0.01),name='W4')
W5 = tf.Variable(tf.random_normal([3, 3, 384, 256], stddev=0.01),name='W5')
keep_prob=tf.placeholder(tf.float32)  

L1 = tf.nn.conv2d(X, W1, strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.relu(L1)
norm1 = tf.nn.lrn(L1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm1')
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.relu(L2)
norm2 = tf.nn.lrn(L2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm2')
L2 = tf.nn.max_pool(L2, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)

L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.relu(L4)

L5 = tf.nn.conv2d(L4, W5, strides=[1, 1, 1, 1], padding='SAME')
L5 = tf.nn.relu(L5)

In [6]:
print(L5.shape[1],L5.shape[2])


14 14


In [7]:
a=int(14)
#SPP 6*6
spp6_ksize=round(a/6+0.49)
spp6_stride=int(a/6)
Spp6 = tf.nn.max_pool(L5,ksize=[1,spp6_ksize,spp6_ksize,1],strides=[1,spp6_stride,spp6_stride,1],padding="SAME")
Spp6_flatten = tf.reshape(Spp6,[-1,256*7*7])

#SPP 3*3
spp3_ksize=round(a/3+0.49)
spp3_stride=int(a/3)
Spp3 = tf.nn.max_pool(L5,ksize=[1,spp3_ksize,spp3_ksize,1],strides=[1,spp3_stride,spp3_stride,1],padding="SAME")
Spp3_flatten = tf.reshape(Spp3,[-1,256*4*4])

#SPP 2*2
spp2_ksize=round(a/2+0.49)
spp2_stride=int(a/2)
Spp2 = tf.nn.max_pool(L5,ksize=[1,spp2_ksize,spp2_ksize,1],strides=[1,spp2_stride,spp2_stride,1],padding="SAME")
Spp2_flatten = tf.reshape(Spp2,[-1,256*2*2])

#SPP 1*1
spp1_ksize=round(a/1+0.49)
spp1_stride=int(a/1)
Spp1 = tf.nn.max_pool(L5,ksize=[1,spp1_ksize,spp1_ksize,1],strides=[1,spp1_stride,spp1_stride,1],padding="SAME")
Spp1_flatten = tf.reshape(Spp1,[-1,256*1*1])

spp_concat=tf.concat([Spp1_flatten,Spp2_flatten,Spp3_flatten,Spp6_flatten],1)

In [8]:
print(Spp1_flatten)
print(Spp2_flatten)
print(Spp3_flatten)
print(Spp6_flatten)
spp_concat

Tensor("Reshape_3:0", shape=(?, 256), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 1024), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 4096), dtype=float32)
Tensor("Reshape:0", shape=(?, 12544), dtype=float32)


<tf.Tensor 'concat:0' shape=(?, 17920) dtype=float32>

In [9]:


#size : 2*2*256
#Fully connected network
Fc6_b=tf.Variable(tf.random_normal([4096]))
Fc7_b=tf.Variable(tf.random_normal([20]))
Fc6_w= tf.get_variable("Fc6_w", shape=[17920, 4096],
     initializer=tf.contrib.layers.xavier_initializer())
Fc7_w= tf.get_variable("Fc7_w", shape=[4096, 20],
     initializer=tf.contrib.layers.xavier_initializer())
FL1 = tf.nn.relu(tf.matmul(spp_concat, Fc6_w) + Fc6_b)
FL1=tf.nn.dropout(FL1,keep_prob=keep_prob)

logits = tf.matmul(FL1, Fc7_w) + Fc7_b




In [10]:
print(FL1)
print(logits)


Tensor("dropout/mul:0", shape=(?, 4096), dtype=float32)
Tensor("add_1:0", shape=(?, 20), dtype=float32)


In [11]:

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [12]:
x_train=[]
for i in range(len(image_data)):
    x_train.append(cv2.resize(image_data[i],(224,224)))
print("complete : %d"%len(x_train))

complete : 17125


In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)


# 정확도를 측정한다.
predictions = tf.argmax(logits, 1)
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
#####################################################training##########################################################################
import time
saver = tf.train.Saver({"W1":W1,"W2":W2,"W3":W3,"W4":W4,"W5":W5})# weight저장
sess = tf.Session()# initialize
sess.run(tf.global_variables_initializer())
#학습시간 측정
start_time = time.time()
training_epochs=71
batch_size=32
saver.restore(sess, "./checkpoint/sppnet_trained_0040.ckpt")#load training data
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
   
    avg_cost = 0
    total_batch = int(len(x_train) / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys =next_batch(batch_size,x_train,y_data)
        #one hot encoding
        batch_ys=np.eye(20)[batch_ys].reshape([-1,20])
        feed_dict = {X: batch_xs, Y: batch_ys,keep_prob:0.1}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        type(c)
        avg_cost += c
        avg_cost /=total_batch
    if(epoch%10==0):
        saver.save(sess, './checkpoint/sppnet_trained_%04d.ckpt' % (epoch))
        print("save ! epoch : ",epoch)
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.8f}'.format(avg_cost))
print('Learning Finished!')
#학습시간 출력
print("time elapsed: {:.2f}s".format(time.time() - start_time))

INFO:tensorflow:Restoring parameters from ./checkpoint/sppnet_trained_0040.ckpt
Learning started. It takes sometime.
save ! epoch :  0
Epoch: 0001 cost = 0.00739247
Epoch: 0002 cost = 0.00436738
Epoch: 0003 cost = 0.00460694
Epoch: 0004 cost = 0.00470738
Epoch: 0005 cost = 0.00487863
Epoch: 0006 cost = 0.00413575
Epoch: 0007 cost = 0.00403094
Epoch: 0008 cost = 0.00394053
Epoch: 0009 cost = 0.00465346
Epoch: 0010 cost = 0.00409997
save ! epoch :  10
Epoch: 0011 cost = 0.00459519
Epoch: 0012 cost = 0.00457546
Epoch: 0013 cost = 0.00512228
Epoch: 0014 cost = 0.00389694
Epoch: 0015 cost = 0.00519322
Epoch: 0016 cost = 0.00461137
Epoch: 0017 cost = 0.00400524
Epoch: 0018 cost = 0.00428642
Epoch: 0019 cost = 0.00435512
Epoch: 0020 cost = 0.00529919
save ! epoch :  20
Epoch: 0021 cost = 0.00412808
Epoch: 0022 cost = 0.00512281
Epoch: 0023 cost = 0.00491268
Epoch: 0024 cost = 0.00388721
Epoch: 0025 cost = 0.00433847
Epoch: 0026 cost = 0.00357951
Epoch: 0027 cost = 0.00454894
Epoch: 0028 cost 

In [23]:
with tf.Session() as sess:
    
    # Initialize v1 since the saver will not.
    W1.initializer.run()
    W2.initializer.run()
    W3.initializer.run()
    W4.initializer.run()
    W5.initializer.run()
    
#     print("W1 : %s" % W1.eval())
#     print("W2 : %s" % W2.eval())
#     print("W3 : %s" % W3.eval())
#     print("W4 : %s" % W4.eval())
#     print("W5 : %s" % W5.eval())
    
    saver.restore(sess, "./checkpoint/sppnet_trained_0070.ckpt")

    print("W1 : %s" % W1.eval())
#     print("W2 : %s" % W2.eval())
#     print("W3 : %s" % W3.eval())
#     print("W4 : %s" % W4.eval())
#     print("W5 : %s" % W5.eval())
    

INFO:tensorflow:Restoring parameters from ./checkpoint/sppnet_trained_0070.ckpt
W1 : [[[[-0.06479301 -0.07185461 -0.06309287 ... -0.0408775  -0.0695301
    -0.05189398]
   [-0.02854208 -0.06887428 -0.05291032 ... -0.07973412 -0.05478656
    -0.05243146]
   [-0.05571016 -0.05655868 -0.05948252 ... -0.03382749 -0.06181584
    -0.04542318]]

  [[-0.04753726 -0.07808983 -0.07109114 ... -0.05055545 -0.05579372
    -0.06345464]
   [-0.0439842  -0.0740972  -0.05081635 ... -0.07162498 -0.05119932
    -0.07166295]
   [-0.05662998 -0.05825284 -0.06054119 ... -0.08441267 -0.05957964
    -0.06321356]]

  [[-0.0527623  -0.06100205 -0.05083722 ... -0.07317793 -0.07730954
    -0.06048567]
   [-0.04201294 -0.05867869 -0.06129504 ... -0.06164214 -0.06542012
    -0.06289586]
   [-0.04071009 -0.06254079 -0.06084927 ... -0.07241112 -0.068876
    -0.06245252]]

  ...

  [[-0.05366499 -0.0610461  -0.06541366 ... -0.04070158 -0.05307648
    -0.04272733]
   [-0.06930017 -0.07060302 -0.0680651  ... -0.0265261 

In [15]:
# #####################################################test##########################################################################
# sess=tf.Session()
# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
# saver.restore(sess, "./checkpoint/sppnet_trained_0030.ckpt")#load training data
# batch_size=1
# length=int(len(x_train)/batch_size)
# avg_acc=0
# for index in range(10):
#     batch_xs, batch_ys = next_batch(batch_size,x_train,y_data)  
#     batch_ys=np.eye(20)[batch_ys].reshape([-1,20])
#     feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.5}
#     acc=sess.run(accuracy,feed_dict=feed_dict)
# #     test_logits=sess.run(predictions,feed_dict)#추정값
# #     test_labels=np.argmax(batch_ys,1)
#     print(" acc : ",acc)
    